In [25]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split # used for splitting training and testing data

torch.manual_seed(1)

In [26]:
plt.figure(figsize=(8,5))

# time steps per batch of data
seq_length = 100

time_steps = np.linspace(0, np.pi, seq_length + 1)

data = pd.read_csv('SectionData.csv')

x = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

<Figure size 576x360 with 0 Axes>

In [27]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, ouput_size, n_layers, hidden_size, drop_prob):
        
        super(NeuralNetwork, self).__init__()
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.drop_prob = drop_prob
        
        self.lstm = nn.LSTM(input_size, hidden_size[0], n_layers, batch_first=True)
        
        self.dropout = nn.Dropout(drop_prob)
        
        # First fully connected layer
        self.fc1 = nn.Linear(hidden_size[1], output_size[0])
        # Second fully connected layer
        self.fc2 = nn.Linear(hidden_size[2], output_size[1])
        
        # Softmax activation
        self.softmax = nn.Softmax()
        
    def forward(self, x, hidden):
        # x(batch_size, seq_length, input_size)
        # hidden(n_layers, batch_size, hidden_dim)
        # r_out(batch_size, time_step, hidden_size)
        batch_size = x.size(0)
        
        # get LSTM outputs
        lstm_out, hidden = self.lstm(x, hidden)
        
        # shape output to be (batch_size*seq_length, hidden_dim)
        lstm_out = lstm_out.view(-1, self.hidden_dim)
        
        # dropout layer
        out = self.dropout(lstm_out)
        
        # get output of first fully connected layer
        out1 = self.fc1(lstm_out)
        
        # get output of second fully connected layer
        out2 = self.fc2(out1)
        
        # softmax activation function
        out2 = self.softmax(out2)
        
        # TODO: might need to reshape the out2
        
        return output, hidden
    
    def init_hidden(self, batch_size):
        '''initialize hidden states'''
        # initialized initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.paramaters()).data
        
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_size).zero_(),
                 weight.new(self.n_layers, batch_size, self.hidden_size).zero_())
        
        return hidden

In [28]:
def preprocess(df):

    # drop ecgNum row
    df = df.drop(["ecgNum"], axis=1)
    # Classify the dependent and independent variables
    X = df.iloc[:, :-1].values
    Y = df.iloc[:, -1].values
    # split the data into train, validate, test
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
    X_train, X_test, X_val, Y_val = train_test_split(X, Y, test_size=0.2, random_state=0)
    
    return X_train, X_val, X_test, Y_train, Y_val, Y_test

In [29]:
preprocess(data)

(array([[-0.04160644, -0.03060149, -0.02387862, ..., -0.04207994,
         -0.04022728, -0.04469989],
        [-0.04249774, -0.0418486 , -0.03964466, ..., -0.05770056,
         -0.0467808 , -0.05952186],
        [-0.00815502,  0.04986301,  0.0463013 , ..., -0.07790836,
         -0.06542077, -0.08319839],
        ...,
        [ 0.02837123,  0.01375099,  0.02311569, ...,  0.05251654,
          0.05867046,  0.06540402],
        [-0.02106233, -0.00757167, -0.00953091, ..., -0.04357452,
         -0.04459247, -0.03766697],
        [ 0.15508868,  0.30135933,  0.29984443, ..., -0.03377062,
         -0.01796592, -0.05416976]]),
 array(['N', 'A', 'N', ..., 'R', 'N', 'V'], dtype=object),
 array([[ 0.09509818,  0.08729745,  0.08196842, ...,  0.08738376,
          0.08252306,  0.08453803],
        [ 0.05997522, -0.02581645, -0.00695863, ...,  0.25887261,
          0.23623245,  0.25365312],
        [-0.03072452, -0.02739705, -0.03340341, ..., -0.03297214,
         -0.02324419, -0.02318459],
        

In [18]:
# size of the input at each time step
# TODO: handle input sizes
input_size = 100
# size of the hidden state and cell state at each time step
hidden_size = [64, 32, 8]
# TODO: ouptut sizes
output_size = [8, 4]
# number of LSTM layers
n_layers = 1
# dropout probability
drop_prob = 0.5

# instantiate the NN
neuralNet = NeuralNetwork(input_size, output_size, n_layers, hidden_size, drop_prob)
print(neuralNet)

NeuralNetwork(
  (lstm): LSTM(100, 64, batch_first=True)
  (fc1): Linear(in_features=32, out_features=8, bias=True)
  (fc2): Linear(in_features=8, out_features=4, bias=True)
  (softmax): Softmax(dim=None)
)


In [19]:
# Mean Squared Error and Adam Optimizer with a learning rate of 0.01
# TODO: play with LR
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(neuralNet.parameters(), lr=0.01)

In [21]:
# train the NN
def train(neuralNet, n_steps, print_every):
    # initialize the hidden state
    hidden = None
    
    X_train, X_val, X_test, Y_train, Y_val, Y_test = preprocess(data)
    
    for batch_i, step in enumerate(range(n_steps)):
        neuralNet.zero_grad()
        
        # output from the neural net
        prediction, hidden = neuralNet(X_train, hidden)
        
        hidden = hiden.data
        
        # calculate the loss
        loss = criterion(prediction, Y_train)
        # zero gradients
        optimizer.zero_grad()
        # backpropogate and update weights
        loss.backward()
        optimizer.step()
        
        # display loss and predictions
        if batch_i%print_every == 0:
            print('Loss ', loss.item())
            plt.plot(time_steps[1:], x, 'r.') #input
            plt.plot(time_steps[1:], predction.data.numpy().flatten(), 'b.') #predictions
            
    return neuralNet

In [22]:
n_steps = 100
print_every = 45

trained_NN = train(neuralNet, n_steps, print_every)

TypeError: 'int' object is not callable